In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# read the csv
current_portfolio = pd.read_csv('U3098324.csv', index_col=None)  

# data cleaning
current_portfolio = current_portfolio.loc[current_portfolio['Open Positions'] == 'Open Positions']
current_portfolio = current_portfolio.loc[current_portfolio['Account'] == 'U3098324/PMP Berlin Quant Group']
current_portfolio = current_portfolio.loc[current_portfolio['Asset Category'] == 'Stocks']
current_portfolio = current_portfolio.reset_index()
current_portfolio = current_portfolio.drop(['Asset Category', 'index', 'Open Positions', 'Header', 'Account', 'Unnamed: 11'], axis=1)
current_portfolio['Cost Basis'] = pd.to_numeric(current_portfolio['Cost Basis']).round().astype(int)
current_portfolio['Value'] = pd.to_numeric(current_portfolio['Value']).round().astype(int)
current_portfolio['Unrealized P/L'] = pd.to_numeric(current_portfolio['Unrealized P/L']).round().astype(int)

# manually insert the exchange rate here, and we use EUR as our base currency here
exchange_rate_table = {'CNH': 0.1272521299, 'EUR':1, 'HKD':0.105952606, 'USD':0.821388969}
exchange_rate = []

current_portfolio.Currency
for i in current_portfolio.Currency:
    exchange_rate.append(exchange_rate_table[i])

current_portfolio['exchange_rate'] = exchange_rate
current_portfolio['Cost Basis(EUR)'] = current_portfolio['Cost Basis']*current_portfolio.exchange_rate
current_portfolio['Weight'] = current_portfolio['Cost Basis'] / current_portfolio['Cost Basis'].sum()

# change the ticker into what IEX can read
ticker = []
ticker_table = {'601006':'601006-CG', 
                'BARb':'BAR', 
                'DOQ':'DOQ-GB', 
                'GLPG':'GLPG-NA', 
                'SAPd':'SAP-GY',
                '1810':'01810-HK',
                '3690':'03690-HK',
                '520':'00520-HK',
                '598':'00598-HK',
                '6098':'06098-HK',
                '6110':'06110-HK',
                '6618':'06618-HK',
                '700':'00700-HK',
                '9633':'09633-HK',
                '968':'00968-HK'
               }

current_portfolio['ticker'] = [ ticker_table.get(item,item) for item in current_portfolio.Symbol ]
current_portfolio.head()


,Currency,Symbol,Quantity,Cost Basis,Close Price,Value,Unrealized P/L,exchange_rate,Cost Basis(EUR),Weight,ticker
0,CNH,601006,10000,65059,6.57,65700,641,0.127252,8278.896319,0.026053,601006-CG
1,EUR,BARb,800,14409,18.47,14776,367,1.000000,14409.000000,0.005770,BAR
2,EUR,DOQ,370,19916,62.6,23162,3246,1.000000,19916.000000,0.007975,DOQ-GB
3,EUR,GLPG,190,15301,89.44,16994,1693,1.000000,15301.000000,0.006127,GLPG-NA
4,EUR,SAPd,400,40362,104.86,41944,1582,1.000000,40362.000000,0.016163,SAP-GY


In [2]:
# Extract historiacl data from IEX
from iexfinance.stocks import get_historical_data
from iexfinance.stocks import Stock
from datetime import datetime
from datetime import timedelta
import os

# Authenticate the IEX with token
os.environ['IEX_API_VERSION'] = 'iexcloud-sandbox'
os.environ['IEX_TOKEN'] = 'Tpk_xxxxxxxxxxx' # put your own token here

# Create a dataframe to store the data
ticker_list = current_portfolio['ticker']
start_date = datetime.now() - timedelta(days = 40)
data = []

df = pd.DataFrame(columns=['symbol', 'fVolume', 'fClose', 'changePercent'])

for s in ticker_list:
    df_extract_1 = get_historical_data(s, start_date).loc[:, ['symbol', 'fVolume', 'fClose', 'changePercent']]
    df = df.append(df_extract_1)

# Change datatype into numeric
df.changePercent = pd.to_numeric(df.changePercent)
df.fClose = pd.to_numeric(df.fClose)
df.fVolume = pd.to_numeric(df.fVolume)
df.head()

,symbol,fVolume,fClose,changePercent
2020-12-14,601006-CG,47139908.0,6.60,-0.0254
2020-12-15,601006-CG,27344889.0,6.81,-0.0062
2020-12-16,601006-CG,25261858.0,6.69,-0.0077
2020-12-17,601006-CG,27777436.0,6.78,0.0048
2020-12-18,601006-CG,37335242.0,6.90,0.0095


In [3]:
# Extract key stats from IEX
df_key_stats = pd.DataFrame(columns=['beta', 'peRatio', 'marketcap'])

for s in ticker_list:
    df_extract_2 = Stock(s).get_key_stats().loc[:, ['beta', 'peRatio', 'marketcap']]
    df_key_stats = df_key_stats.append(df_extract_2)

df_key_stats # it's seem like IEX doesn't have full data regarding to beta, P/E and MarketCap

,beta,peRatio,marketcap
601006-CG,0,0,98511898181
BAR,0.10194,0,1149502418
DOQ-GB,0,0,606100345
GLPG-NA,0,0,6205438181
SAP-GY,0,0,262467446388
...,...,...,...
U,0,-305.723,41092865889
VBR,1.21127,0,19057006696
VTV,1.03899,0,64075781440
WMT,0.532842,20.9147,417905005561


In [4]:
# Add mean and standard deviation into dataframe

mean = list(df.groupby(['symbol']).changePercent.mean().reindex(ticker_list))
SD = list(df.groupby(['symbol']).changePercent.std().reindex(ticker_list))
current_portfolio['mean'] = mean
current_portfolio['Standard Deviation'] = SD
current_portfolio.head()

,Currency,Symbol,Quantity,Cost Basis,Close Price,Value,Unrealized P/L,exchange_rate,Cost Basis(EUR),Weight,ticker,mean,Standard Deviation
0,CNH,601006,10000,65059,6.57,65700,641,0.127252,8278.896319,0.026053,601006-CG,-0.001510,0.008577
1,EUR,BARb,800,14409,18.47,14776,367,1.000000,14409.000000,0.005770,BAR,0.000723,0.011584
2,EUR,DOQ,370,19916,62.6,23162,3246,1.000000,19916.000000,0.007975,DOQ-GB,0.007592,0.034250
3,EUR,GLPG,190,15301,89.44,16994,1693,1.000000,15301.000000,0.006127,GLPG-NA,-0.001652,0.041983
4,EUR,SAPd,400,40362,104.86,41944,1582,1.000000,40362.000000,0.016163,SAP-GY,0.002160,0.012295


In [15]:
# Calculate the portfolio standard deviation
import math

df_return = pd.DataFrame(columns=ticker_list)

for s in ticker_list:
    df_return[s] = df.loc[df.symbol == s].changePercent

cov_matrix = df_return.cov().values

weight_vector = current_portfolio.Weight.to_numpy()
weight_matirx = np.outer(weight_vector, weight_vector)

variance = np.multiply(cov_matrix, weight_matirx).sum()
portfolio_sigma = math.sqrt(variance)

print(portfolio_sigma)

0.01337129109105298


In [21]:
# Risk Accessment
df_risk = current_portfolio[['ticker','Cost Basis(EUR)','Weight', 'Standard Deviation']]
df_risk['position_risk_value'] = df_risk['Cost Basis(EUR)']*df_risk['Standard Deviation']
df_risk['position_risk_weight'] = df_risk['position_risk_value'] / df_risk['position_risk_value'].sum()
df_risk

,ticker,Cost Basis(EUR),Weight,Standard Deviation,position_risk_value,position_risk_weight
0,601006-CG,8278.896319,0.026053,0.008577,71.012039,0.001818
1,BAR,14409.000000,0.005770,0.011584,166.912357,0.004273
2,DOQ-GB,19916.000000,0.007975,0.034250,682.132112,0.017464
3,GLPG-NA,15301.000000,0.006127,0.041983,642.386985,0.016447
4,SAP-GY,40362.000000,0.016163,0.012295,496.267983,0.012706
...,...,...,...,...,...,...
65,U,18163.374271,0.008855,0.036794,668.311785,0.017111
66,VBR,13341.821023,0.006505,0.013519,180.372410,0.004618
67,VTV,13239.968791,0.006455,0.008221,108.851263,0.002787
68,WMT,19981.108060,0.009741,0.008341,166.657460,0.004267


In [23]:
# calculate marginal risk contribution
w = np.matrix(w)
V = cov_matrix

# Marginal Risk Contribution
MRC = V*w.T

# Risk Contribution
df_risk['Marginal_Risk'] = np.multiply(MRC,w.T)/portfolio_sigma
df_risk['Risk_contirbution'] = df_risk['Weight'] * df_risk['Marginal_Risk']
df_risk['money_Risk'] = df_risk['Risk_contirbution']*df_risk['Cost Basis(EUR)'].sum()
df_risk['risk_weight'] = df_risk['money_Risk']/df_risk['money_Risk'].sum()
df_risk

,ticker,Cost Basis(EUR),Weight,Standard Deviation,position_risk_value,position_risk_weight,Marginal_Risk,Risk_contirbution,money_Risk,risk_weight
0,601006-CG,8278.896319,0.026053,0.008577,71.012039,0.001818,1.060288e-07,2.762367e-09,0.003250,0.000006
1,BAR,14409.000000,0.005770,0.011584,166.912357,0.004273,1.785029e-05,1.029981e-07,0.121168,0.000205
2,DOQ-GB,19916.000000,0.007975,0.034250,682.132112,0.017464,-1.673564e-05,-1.334734e-07,-0.157020,-0.000266
3,GLPG-NA,15301.000000,0.006127,0.041983,642.386985,0.016447,5.147024e-05,3.153742e-07,0.371010,0.000628
4,SAP-GY,40362.000000,0.016163,0.012295,496.267983,0.012706,5.706416e-06,9.223298e-08,0.108504,0.000184
...,...,...,...,...,...,...,...,...,...,...
65,U,18163.374271,0.008855,0.036794,668.311785,0.017111,1.264695e-04,1.119912e-06,1.317476,0.002231
66,VBR,13341.821023,0.006505,0.013519,180.372410,0.004618,2.184903e-05,1.421179e-07,0.167189,0.000283
67,VTV,13239.968791,0.006455,0.008221,108.851263,0.002787,1.131816e-05,7.305739e-08,0.085946,0.000146
68,WMT,19981.108060,0.009741,0.008341,166.657460,0.004267,1.041104e-05,1.014180e-07,0.119309,0.000202
